In [64]:
### Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd
from keras.callbacks import EarlyStopping

##Load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
##save to a file
with open('hamlet.txt','w') as file:
    file.write(data)


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\19408\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [65]:
## Data preprocessing
import numpy as np
#from tensorflow.keras.preprocessing import Tokenizer
##from tensorflow.keras.preprocessing.sequence import pad_sequences
# Use the new import path
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

from sklearn.model_selection import train_test_split


##load the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()
## Tokenize the text - creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words


4818

In [66]:
## Create input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [67]:
## Pad sequences 
max_sequence_len=max([len(x) for x in input_sequences])

In [68]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [69]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [70]:
## create predictors and labels
import tensorflow as tf
X,y=input_sequences[:,:-1],input_sequences[:,-1]


In [71]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [72]:
##split the datae\set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

##Define early stopping'
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [73]:
## Train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout


#Define the modelk
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

##COmpile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_6 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_2 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               100400    
                                                                 
 dense_2 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [74]:
##Train the model
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),verbose=1)


Epoch 1/50
644/644 [==============================] - 19s 23ms/step - loss: 6.9122 - accuracy: 0.0329 - val_loss: 6.7320 - val_accuracy: 0.0315
Epoch 2/50
644/644 [==============================] - 15s 23ms/step - loss: 6.4750 - accuracy: 0.0379 - val_loss: 6.8185 - val_accuracy: 0.0412
Epoch 3/50
644/644 [==============================] - 16s 25ms/step - loss: 6.3380 - accuracy: 0.0447 - val_loss: 6.8545 - val_accuracy: 0.0484
Epoch 4/50
644/644 [==============================] - 16s 25ms/step - loss: 6.1845 - accuracy: 0.0527 - val_loss: 6.8940 - val_accuracy: 0.0480
Epoch 5/50
644/644 [==============================] - 15s 24ms/step - loss: 6.0364 - accuracy: 0.0573 - val_loss: 6.9269 - val_accuracy: 0.0521
Epoch 6/50
644/644 [==============================] - 16s 25ms/step - loss: 5.8931 - accuracy: 0.0656 - val_loss: 6.9860 - val_accuracy: 0.0544
Epoch 7/50
644/644 [==============================] - 16s 24ms/step - loss: 5.7582 - accuracy: 0.0731 - val_loss: 7.0653 - val_accuracy:

In [83]:
## Function to predict next word
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len):]  # Ensure the sequence length matches max_sequence_len
    token_list=pad_sequences([token_list], maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word 
    return None

In [84]:
input_text="To be or not to be"
print(f"Input text is :{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction : {next_word}")

Input text is :To be or not to be
Next word prediction : too


In [86]:
##save the model

model.save("next_word_lstm.h5")


##save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [87]:
input_text="To be bad is better than"
print(f"Input text is :{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction : {next_word}")

Input text is :To be bad is better than
Next word prediction : a
